# i still hate this so much

i kinda never liked the syms thing from matlab, probably because i dont understand!!!!

why do some fns declare like a variable, but some others you have to `function [o] = function(i)` ????  
AND THEN WTF IS @(x)?????????


## here is an explanation.

imagine this. you import sympy:


In [ ]:
pip install sympy

In [1]:
import sympy
import numpy

sympy provides a Symbol class:


In [2]:
x_lda = sympy.Symbol("x")
y_lda = sympy.Symbol("y")

now here is funny: what you just made are symbols. it has proper `__magic__` functions that do things different than a number. check this out:


In [3]:
f_lda = x_lda**3 - x_lda - 2
f_lda

x**3 - x - 2

fancy ass renders from jupyter aside, this thing is now a sympy something construct. it is NOT a function. it is a variable. it is... certainly something.

### symbolic computation

[sympy tutorial][st] explains: [mathematical expressions with unevaluated variables are left in symbolic form.][expl]

i explain: you do math on paper: you dont write the decimal number out, you just write the final answer as $\frac{\sqrt 3 + 1}{2}$

so the sympy does the same: it writes that instead of idfk `1.3660254038`

[st]: https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html
[expl]: https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html#:~:text=mathematical%20expressions%20with%20unevaluated%20variables%20are%20left%20in%20symbolic%20form.


In [4]:
root_idk = sympy.factor((sympy.sqrt(3) + 1) / 2)

numpy.testing.assert_approx_equal(root_idk.evalf(), (3**0.5 + 1) / 2)  # passes!
root_idk

(1 + sqrt(3))/2

you see in mat lab they pass this kinda sympy symbolic ahh structure Literally everywhere. i fear that. that is why i dont like this.

from [matlab]:

[matlab]: https://www.mathworks.com/help/symbolic/create-symbolic-numbers-variables-and-expressions.html#:~:text=Solve%20the%20quadratic,f%20%3D%3D%200%2Cx

> Solve the quadratic equation f=0 for x by using solve.
>
> ```matlab
> x_0 = solve(f == 0,x)
> ```

well you can do that with sympy too:


In [5]:
for i in sympy.solve(f_lda):
    print(i, i.evalf())

(-1/2 - sqrt(3)*I/2)*(sqrt(78)/9 + 1)**(1/3) + 1/(3*(-1/2 - sqrt(3)*I/2)*(sqrt(78)/9 + 1)**(1/3)) -0.760689853402284 - 0.857873626595179*I
1/(3*(-1/2 + sqrt(3)*I/2)*(sqrt(78)/9 + 1)**(1/3)) + (-1/2 + sqrt(3)*I/2)*(sqrt(78)/9 + 1)**(1/3) -0.760689853402284 + 0.857873626595179*I
1/(3*(sqrt(78)/9 + 1)**(1/3)) + (sqrt(78)/9 + 1)**(1/3) 1.52137970680457


god damn

### no need (sometimes)

well thats cool and all if the thing doesnt start to rely on numerical solves: check [this][ssp] out

[ssp]: https://docs.sympy.org/latest/modules/solvers/solvers.html#sympy.solvers.solvers.solve:~:text=Currently%20supported%3A

and then when that happen youd rather:


In [6]:
try:
    import scipy.optimize
except:
    # %pip install scipy
    get_ipython().run_line_magic(
        "pip", "install scipy"
    )  # prev line resolves to this so i use # why is pylance ass
    import scipy.optimize

In [7]:
f_lda_py = lambda x: x**3 - x - 2
scipy_res = scipy.optimize.root(f_lda_py, 1)
scipy_res.x[0]  # so much notation to extract the one real root lmfao

np.float64(1.5213797068045676)

look: just make a python function? and use an nsolve if you plan to use nsolve anyway? why bother yourself with symbols when you never cares about symbols in your answer?

what if i never had a chance to get a concrete answer like this: $\frac{1}{3\left(\frac{\sqrt{78}}{9} + 1\right)^{\frac{1}{3}}} + \left(\frac{\sqrt{78}}{9} + 1\right)^{\frac{1}{3}}$? and i dont want to see such shitass anyway.  
a number like this: `1.5213797068045676` works way better


## bisection (pure python cuz i cant work with modules)

bisection step is:

step 1: check low. high, mid

step 1.5: know that high is higher than 0 and low is lower than 0

step 2: if mid is low then mid moves to low  
and if mid is high then mid moves to high

step 3: step 1


In [8]:
from typing import Callable
from numbers import Real
from math import isclose


def bis(
    f: Callable[[Real], Real],
    low: Real,
    high: Real,
    *,
    rel_tol: Real = 1e-9,
    abs_tol: Real = 0,
    max_loop: int = 100,
) -> tuple[Real, int]:
    ldaprx = lambda a, b: isclose(a, b, rel_tol=rel_tol, abs_tol=abs_tol)
    steps = 0
    fl = f(low)
    fh = f(high)
    #     raise ValueError(f"at {low =} f(x) is {fl}, at {high =} f(x) is {fh} which has the same sign")

    if ldaprx(0, fl):
        return (low, 0)
    elif ldaprx(0, fh):
        return (high, 0)

    while (steps := steps + 1) and (max_loop := max_loop - 1) + 1:
        mid = (low + high) / 2
        match f(mid):
            case x if ldaprx(0, x):
                return (mid, steps)
            case x if x < 0:
                low = mid
            case x if x > 0:
                high = mid
    return (mid, steps)

In [9]:
bis(f_lda_py, 1, 2)

(1.5213797068045676, 52)

52 loops! damn
